In [403]:
import numpy as np
from numpy import inf
from random import choices

from utilities import read_instance,  visualize, instance

In [405]:
n = 10
name, capacity, node_data = read_instance("../gehring instances/1000/C1_10_1.TXT")
ins = instance(name, capacity, node_data, n)
ins.capacity = 20

In [406]:
D = ins.cost
D[D == 0] = inf
p = 1/D

In [407]:
# Q = 3
# nodes[(load[gate[nodes]] <= Q -1) & (pred[nodes] > -1)]

# Procedimiento 1

In [408]:
def explorar1(ins):
    nodes = np.array(range(n + 1), dtype = int)
    pred = np.ones(n+1, dtype = int) * -1
    gate = np.zeros(n+1, dtype = int)
    load = np.zeros(n+1, dtype = int)
    arrival = np.zeros(n+1)

    itree = [0]
    nodes_left = set(nodes) - set([0])
    nodes_left

    Q = 3
    earliest = np.array(ins.earliest)
    latest = np.array(ins.latest)
    
    cost = 0
    while len(nodes_left) > 0:
        candidates = []
        for k in itree:
            nleft = list(nodes_left)
            prob = p[k, nleft]
            j  = choices(nleft, weights =  prob, k = 1)[0]
            candidates.append((k,j))

        m = len(itree)
        new_prob = np.zeros(m)
        for i, (k,j) in enumerate(candidates):
            new_prob[i] = p[k,j]

        i  = choices(range(m), weights =  new_prob, k = 1)[0]
        kk,jj = candidates[i]

        itree.append(jj)
        nodes_left.remove(jj)

        pred[jj] = kk

        cost += D[kk,jj]
        if gate[kk] == 0:
            gate[jj] = jj
        else:
            gate[jj] = gate[kk]

        load[gate[jj]] += 1
        arrival[jj] = arrival[kk] + D[kk,jj]
    
        if not arrival[jj] >= earliest[jj]:
            arrival[jj] = earliest[jj]

# Procedimiento 2

In [409]:
def explorar2(ins):
    nodes = np.array(range(n + 1), dtype = int)
    pred = np.ones(n+1, dtype = int) * -1
    gate = np.zeros(n+1, dtype = int)
    load = np.zeros(n+1, dtype = int)
    arrival = np.zeros(n+1)

    itree = [0]
    nodes_left = set(nodes) - set([0])
    nodes_left

    Q = 3
    earliest = np.array(ins.earliest)
    latest = np.array(ins.latest)
    
    cost = 0
    while len(nodes_left) > 0:
        nleft = list(nodes_left)
        prob = p[itree,:][:, nleft].sum(axis = 1)

        kk  = choices(itree, weights = prob, k  =1)[0]

        prob = p[kk, nleft]
        jj = choices(nleft, weights =  prob, k = 1)[0]

    # ver si tiene sentido meter esto en una función de actualización para acortar el código
        itree.append(jj)
        nodes_left.remove(jj)

        pred[jj] = kk

        cost += D[kk,jj]
        if gate[kk] == 0:
            gate[jj] = jj
        else:
            gate[jj] = gate[kk]

        load[gate[jj]] += 1
        arrival[jj] = arrival[kk] + D[kk,jj]
    
        if not arrival[jj] >= earliest[jj]:
            arrival[jj] = earliest[jj]

# print(j,k)

# Procedimiento 3

In [410]:
def explorar3(ins):
    start = perf_counter()
    Q = ins.capacity
    n = ins.n
    nodes = np.array(range(n), dtype = int)
    pred = np.ones(n, dtype = int) * -1
    gate = np.zeros(n, dtype = int)
    load = np.zeros(n, dtype = int)
    arrival = np.zeros(n)

    next = np.ones((n,n))

    itree = [0]
    nodes_left = set(nodes) - set([0])
    nodes_left

    earliest = np.array(ins.earliest)
    latest = np.array(ins.latest)

    cost = 0
    while len(nodes_left) > 0:
        p[(load[gate][:,None] + next > Q) | (arrival[:,None] + D > latest)] = 0 # establecer filtro
        nleft = list(nodes_left)
        prob = p[itree,:][:, nleft]
        filas = len(itree)
        columnas = len(nodes_left)
        m = filas * columnas
        i = choices(range(m), weights = prob.reshape(m), k = 1)[0]
        k,j = i%filas, i//filas

        kk = itree[k]
        jj = nleft[j]

    # ver si tiene sentido meter esto en una función de actualización para acortar el código
        itree.append(jj)
        nodes_left.remove(jj)

        pred[jj] = kk

        cost += D[kk,jj]
        if gate[kk] == 0:
            gate[jj] = jj
        else:
            gate[jj] = gate[kk]

        load[gate[jj]] += 1
        arrival[jj] = arrival[kk] + D[kk,jj]
    
        if not arrival[jj] >= earliest[jj]:
            arrival[jj] = earliest[jj]

    time = perf_counter() - start
    return cost, time, None, None

# Procedimiento 4

In [411]:
def explorar4(ins):
    start = perf_counter()
    Q = ins.capacity
    n = ins.n
    nodes = np.array(range(n), dtype = int)
    pred = np.ones(n, dtype = int) * -1
    gate = np.zeros(n, dtype = int)
    load = np.zeros(n, dtype = int)
    arrival = np.zeros(n)

    next = np.ones((n,n))

    itree = [0]
    nodes_left = set(nodes) - set([0])
    nodes_left

    earliest = np.array(ins.earliest)
    latest = np.array(ins.latest)

    cost = 0
    while len(nodes_left) > 0:
        p[(load[gate][:,None] + next > Q) | (arrival[:,None] + D > latest)] = 0 # establecer filtro
        nleft = list(nodes_left)
        prob = p[itree,:][:, nleft].sum(axis = 1)
        kk  = np.random.choice(itree, p = prob/prob.sum(), size  =1)[0]

        prob = p[kk, nleft]
        jj = np.random.choice(nleft, p = prob/prob.sum(), size  =1)[0]

    # ver si tiene sentido meter esto en una función de actualización para acortar el código
        itree.append(jj)
        nodes_left.remove(jj)

        pred[jj] = kk

        cost += D[kk,jj]
        if gate[kk] == 0:
            gate[jj] = jj
        else:
            gate[jj] = gate[kk]
            
        load[gate[jj]] += 1
        arrival[jj] = arrival[kk] + D[kk,jj]

        if not arrival[jj] >= earliest[jj]:
            arrival[jj] = earliest[jj]
    time = perf_counter() - start
    return cost, time, None, None


In [428]:
explorar4(ins)

(1767.0485346133057, 0.0032478999983140966, None, None)

In [424]:
explorar3(ins)

(2971.9583754260134, 0.0021014999983890448, None, None)